In [ ]:
!py -m pip install ibm_watson
!py -m pip install ttkthemes

In [5]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import requests

def translate_text(source_lang, target_lang, text_to_translate):
    url = "https://api.au-syd.language-translator.watson.cloud.ibm.com/instances/b541cc9f-5d84-42e9-876c-14b3a5b597c0/v3/translate?version=2018-05-01"
    auth = ("apikey", "ApZ46Cv5jQTNaYZHrA7RGer-XiZxjNuznv7iYtS49DnD")
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
    data = {
        "text": [text_to_translate],
        "source": source_lang,
        "target": target_lang
    }
    response = requests.post(url, headers=headers, auth=auth, json=data)
    if response.status_code == 200:
        return response.json()["translations"][0]["translation"]
    else:
        return None

def load_languages(file_path):
    language_codes = {}
    with open(file_path, "r") as file:
        for line in file:
            parts = line.strip().split(" ", 1)
            if len(parts) != 2:
                print(f"Error: Invalid line format - {line}")
                continue
            language, code = parts
            language_codes[language.strip()] = code.strip()
    return language_codes

def translate():
    source_lang_name = source_lang_combobox.get()
    target_lang_name = target_lang_combobox.get()
    source_lang = language_codes.get(source_lang_name)
    target_lang = language_codes.get(target_lang_name)
    text_to_translate = left_text.get("1.0", tk.END).strip()

    if text_to_translate:
        translated_text = translate_text(source_lang, target_lang, text_to_translate)
        if translated_text:
            right_text.delete("1.0", tk.END)
            right_text.insert(tk.END, translated_text)
        else:
            right_text.delete("1.0", tk.END)
            right_text.insert(tk.END, "Error: Translation Failed")

def cancel():
    left_text.delete("1.0", tk.END)
    right_text.delete("1.0", tk.END)

def open_translate_document_window():
    global translate_doc_window
    translate_doc_window = tk.Toplevel(root)
    translate_doc_window.title("Translate Document")

    languages = sorted(language_codes.keys())

    source_lang_label = ttk.Label(translate_doc_window, text="Source Language:")
    source_lang_label.grid(row=0, column=0, padx=10, pady=5)
    global source_lang_combobox
    source_lang_combobox = ttk.Combobox(translate_doc_window, values=languages, state="readonly", width=30, font="Helvetica 14")
    source_lang_combobox.grid(row=0, column=1, padx=10, pady=5)

    target_lang_label = ttk.Label(translate_doc_window, text="Target Language:")
    target_lang_label.grid(row=1, column=0, padx=10, pady=5)
    global target_lang_combobox
    target_lang_combobox = ttk.Combobox(translate_doc_window, values=languages, state="readonly", width=30, font="Helvetica 14")
    target_lang_combobox.grid(row=1, column=1, padx=10, pady=5)

    import_button = CustomButton(translate_doc_window, text="Import Document", command=lambda: import_document(translate_doc_window))
    import_button.grid(row=2, column=0, columnspan=2, padx=10, pady=5)

    global translated_text_textarea
    translated_text_textarea = tk.Text(translate_doc_window, height=10, width=50, font="Helvetica 12")
    translated_text_textarea.grid(row=3, column=0, columnspan=2, padx=10, pady=5)

def import_document(window):
    global document_path
    document_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
    if document_path:
        with open(document_path, "r", encoding="utf-8") as file:
            text_to_translate = file.read()
            messagebox.showinfo("Import Document", f"Document imported: {document_path}")
            target_lang_name = target_lang_combobox.get()  # Get the selected target language
            target_lang = language_codes.get(target_lang_name)  # Get the language code for the selected target language
            translated_text = translate_text(source_lang_combobox.get(), target_lang, text_to_translate)  # Translate the text
            if translated_text:
                translated_text_textarea.delete("1.0", tk.END)
                translated_text_textarea.insert(tk.END, translated_text)
            else:
                translated_text_textarea.delete("1.0", tk.END)
                translated_text_textarea.insert(tk.END, "Error: Translation Failed")
            

def translate_document():
    open_translate_document_window()

class CustomButton(ttk.Button):
    def __init__(self, master=None, **kw):
        ttk.Button.__init__(self, master, **kw)
        self.configure(style='Custom.TButton')

root = tk.Tk()
root.title("IBM Watson Translator")

title_label = ttk.Label(root, text="IBM Watson Translator", font=("Helvetica", 18), foreground="#0f62fe", padding=(20, 10))
title_label.pack(fill=tk.X)

drag_drop_frame = ttk.Frame(root)
drag_drop_frame.pack(fill=tk.X, pady=10)

left_frame = ttk.Frame(drag_drop_frame)
left_frame.pack(side=tk.LEFT, padx=10, fill=tk.Y, expand=True)

language_codes = load_languages("languages.txt")
languages = sorted(language_codes.keys())

source_lang_label = ttk.Label(left_frame, text="Source Language:")
source_lang_label.pack(side=tk.TOP, padx=10, pady=5)
global source_lang_combobox
source_lang_combobox = ttk.Combobox(left_frame, values=languages, state="readonly", width=30, font="Helvetica 14")
source_lang_combobox.pack(side=tk.TOP, padx=10, pady=5)

left_text = tk.Text(left_frame, height=10, width=40, font="Helvetica 14", highlightthickness=2, highlightbackground="#0f62fe",
                    spacing3=12, borderwidth=0, relief="solid")
left_text.pack(side=tk.BOTTOM, padx=10, fill=tk.BOTH, expand=True)

right_frame = ttk.Frame(drag_drop_frame)
right_frame.pack(side=tk.RIGHT, padx=10, fill=tk.Y, expand=True)

target_lang_label = ttk.Label(right_frame, text="Target Language:")
target_lang_label.pack(side=tk.TOP, padx=10, pady=5)
global target_lang_combobox
target_lang_combobox = ttk.Combobox(right_frame, values=languages, state="readonly", width=30, font="Helvetica 14")
target_lang_combobox.pack(side=tk.TOP, padx=10, pady=5)

right_text = tk.Text(right_frame, height=10, width=40, font="Helvetica 14", highlightthickness=2, highlightbackground="#0f62fe",
                     spacing3=12, borderwidth=0, relief="solid")
right_text.pack(side=tk.BOTTOM, padx=10, fill=tk.BOTH, expand=True)

button_frame = ttk.Frame(root)
button_frame.pack(pady=10, fill=tk.X)

cancel_button = CustomButton(button_frame, text="Cancel", command=cancel)
cancel_button.grid(row=0, column=0, padx=10)
cancel_button.configure(style='Custom.TButton')

translate_button = CustomButton(button_frame, text="Translate", command=translate)
translate_button.grid(row=0, column=1, padx=10)
translate_button.configure(style='Custom.TButton')

translate_doc_button = CustomButton(button_frame, text="Translate Document", command=translate_document)
translate_doc_button.grid(row=0, column=2, padx=10)
translate_doc_button.configure(style='Custom.TButton')

# Custom Style for Buttons
style = ttk.Style()
style.configure('Custom.TButton', foreground='white', font=('Helvetica', 12), borderwidth=0, relief="flat")
style.map('Custom.TButton',
          foreground=[('active', 'white')],
          background=[('active', '#4a90e2')])

# Add rounded corners to buttons
style.theme_create("customButtonTheme", parent="alt", settings={
    "TButton": {
        "configure": {"relief": "flat", "borderwidth": 0},
        "map": {
            "background": [("active", "#4a90e2")],
            "foreground": [("active", "white")]
        }
    }
})
style.theme_use("customButtonTheme")

root.mainloop()